In [ ]:
import g2o
import numpy as np

tag_ids = list(range(5))
tag_size = 0.14

optimizer = g2o.SparseOptimizer()
solver = g2o.BlockSolverSE3(g2o.LinearSolverCholmodSE3())
solver = g2o.OptimizationAlgorithmLevenberg(solver)
optimizer.set_algorithm(solver)

focal_length = 1000
principal_point = (320, 240)

# setup artificial observations of tag corners for testing
cam = g2o.CameraParameters(focal_length, principal_point, 0)
cam.set_id(0)

optimizer.add_parameter(cam)

vertex_counter = 0
true_3d_points = np.array([[-tag_size/2,-tag_size/2,1], [tag_size/2,-tag_size/2,1], [tag_size/2,tag_size/2,1], [-tag_size/2,tag_size/2,1]])
fixed_vertex = None

for idx, tag_id in enumerate(tag_ids):
    this_vertex_true_pose = g2o.SE3Quat(np.identity(3), [0.1*idx, 0, 0])
    v1 = g2o.VertexSE3Expmap()
    v1.set_id(vertex_counter)
    vertex_counter += 1
    v1.set_estimate(this_vertex_true_pose)

    optimizer.add_vertex(v1)

    if fixed_vertex is None:
        fixed_vertex = v1
        v1.set_fixed(True)
        fixed_pose = this_vertex_true_pose
    else:
        e = g2o.EdgeSE3Expmap()
        e.set_vertex(0, fixed_vertex)
        e.set_vertex(1, v1)
        relative_pose = fixed_pose.inverse()*this_vertex_true_pose
        e.set_measurement(relative_pose)
        e.set_information(np.identity(6))
        optimizer.add_edge(e)

    # create an anchor vertex
    v_anchor = g2o.VertexSE3Expmap()
    v_anchor.set_id(vertex_counter)
    vertex_counter += 1
    v_anchor.set_estimate(g2o.SE3Quat(np.identity(3), [0.1*idx, 0, 0]))
    optimizer.add_vertex(v_anchor)

    for corner_idx in range(4):
        corner_vertex = g2o.VertexSBAPointXYZ()
        corner_vertex.set_id(vertex_counter)
        vertex_counter += 1
        corner_vertex.set_fixed(True)
        optimizer.add_vertex(corner_vertex)
        corner_vertex.set_estimate(true_3d_points[corner_idx])
        e = g2o.EdgeProjectPSI2UV()
        e.set_vertex(0, corner_vertex)
        e.set_vertex(1, v1)
        e.set_vertex(2, v_anchor)
        e.set_measurement([0.0, 0.0])
        e.set_information(np.identity(2))
        e.set_parameter_id(0, 0)
        optimizer.add_edge(e)

print('num vertices:', len(optimizer.vertices()))
print('num edges:', len(optimizer.edges()))

print('Performing full BA:')
optimizer.initialize_optimization()
optimizer.set_verbose(True)
optimizer.optimize(1)
print("ending chi2", optimizer.chi2())

In [2]:
optimizer.vertices()[15].estimate().translation()

array([ 2.98817457e-01, -2.72254461e-04,  4.84492575e-04])

In [2]:
print(true_3d_points)

[[-0.07 -0.07  1.  ]
 [ 0.07 -0.07  1.  ]
 [ 0.07  0.07  1.  ]
 [-0.07  0.07  1.  ]]
